### Cap Valuation using Black76
We will price a Swaption using Black76 model.\
Let us consider that there is a 1x2 swaption. The swap has semi annual frequency. Assume a notional of $1M.\
At Time T0, the yield curve is as below:

Tenor 0.5 = 5.00% \
Tenor 1.0 = 6.00% \
Tenor 1.5 = 6.75% \
Tenor 2.0 = 7.25% \
Tenor 2.5 = 7.50% \
Tenor 3.0 = 7.60% 

Calculate the Cap price using Black76 Model assuming a constant vol of 2% and a fixed rate of 8%

In [78]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [79]:
def CapCalculator(YieldMatrix,Alpha,Beta,Tau):
    #Find the Index of the first and last reset date
    startResetIndex = YieldMatrix['Tenor'].tolist().index(Alpha+Tau)-1
    EndResetIndex = YieldMatrix['Tenor'].tolist().index(Beta)

    YieldMatrix[['F']] = 0 #Forward Rate Column
    YieldMatrix[['P']] = 0 #Zero Coupon Bond Value

    YieldMatrix[['d1']] = 0 #d1 of Bl76 Model
    YieldMatrix[['d2']] = 0 #d2 of Bl76 Model

    YieldMatrix[['C']] = 0 #Price of the Caplet as per Bl76 Model

    for i in range(startResetIndex,EndResetIndex,):
        R2T2 = ((YieldMatrix.loc[i+1,'Yield'] * YieldMatrix.loc[i+1,'Tenor'])) 
        R1T1 = ((YieldMatrix.loc[i,'Yield'] * YieldMatrix.loc[i,'Tenor']))
        T2MinusT1 = (YieldMatrix.loc[i+1,'Tenor']) - (YieldMatrix.loc[i,'Tenor'])
        YieldMatrix.loc[i,'F'] = (R2T2 - R1T1)/T2MinusT1 #Forward Rate
    
        YieldMatrix.loc[i,'P'] = np.exp(-YieldMatrix.loc[i+1,'Yield'] * YieldMatrix.loc[i+1,'Tenor']) #Zero Coupon Bond Value
    
        YieldMatrix.loc[i,'d1'] = (np.log(YieldMatrix.loc[i,'F']/k) + (0.5 * Vol**2) * YieldMatrix.loc[i,'Tenor'])/(Vol * np.sqrt(YieldMatrix.loc[i,'Tenor']))
        YieldMatrix.loc[i,'d2'] = YieldMatrix.loc[i,'d1'] - Vol * np.sqrt(YieldMatrix.loc[i,'Tenor'])
        
        #Price of the Caplet as per Bl76 Model
        YieldMatrix.loc[i,'C'] = N * Tau * YieldMatrix.loc[i,'P'] * (YieldMatrix.loc[i,'F'] * norm.cdf(YieldMatrix.loc[i,'d1']) - k * norm.cdf(YieldMatrix.loc[i,'d2']))
    
        CapValue = np.sum(YieldMatrix[['C']])
    
    return(CapValue)

In [80]:
Alpha = 1.0
Beta = 3.0
Vol = 0.02
Tau = 0.5 #Semiannual frequency
N = 1000000 #$1 million Notional
f = (Beta-Alpha)/Tau #Number of swap payments
k = 0.08 #Assume Fixed Rate is 8%

YieldMatrix = pd.DataFrame([[0.5,0.05],[1.0,0.06],[1.5,0.0675],[2.0,0.0725],[2.5,0.075],[3.0,0.076]])
YieldMatrix = YieldMatrix.rename(columns = {0:'Tenor',1:'Yield'})

CapValue = CapCalculator(YieldMatrix,Alpha,Beta,Tau)
print(CapValue)

C    7105.365883
dtype: float64
